# vector_alg_development

The purpose of this notebook is to develop practice versions of
an algorithm to convert strings to vectors in a way that
converts string edit distance to Euclidean distance.

## Imports

In [37]:
# Standard library imports
import random

# Third party libraries
import faiss
import numpy as np

## 1. Simply respect replacement distance

The first vectorization should give us exactly
edit distance in Euclidean distance. This is actually
impossible with one coordinate per base, so we'll
use 3 (you need n-1 dimenions for an alphabet of size n).

In [2]:
one_over_root_2 = 1 / np.sqrt(2)

In [3]:
one_over_root_2

0.7071067811865475

In [4]:
# These points are the vertices of a regular tetrahedron centered at the origin.
# I found these values here:
# https://en.wikipedia.org/wiki/Tetrahedron#Coordinates_for_a_regular_tetrahedron
base_vecs = [
    np.array([ 1,  0,  one_over_root_2]),
    np.array([-1,  0,  one_over_root_2]),
    np.array([ 0,  1, -one_over_root_2]),
    np.array([ 0, -1, -one_over_root_2])
]

In [5]:
def dist(a, b):
    return np.linalg.norm(a - b)

In [11]:
# Confirm the base vectors are equidistant.
for i in range(4):
    for j in range(i + 1, 4):
        print(f'{dist(base_vecs[i], base_vecs[j]):.2f}')

2.00
2.00
2.00
2.00
2.00
2.00


In [12]:
np.concatenate([[1, 2], [3, 4]])

array([1, 2, 3, 4])

In [14]:
list('ACGT').index('C')

1

In [15]:
# Convert a string to a vector.
# Each character in s will be come three coordinates in the output string.
# It's expected that s is a string from the alphabet 'ACGT'.
def vec1(s):
    return np.concatenate([base_vecs[list('ACGT').index(let)] for let in s])

In [20]:
vecs = [vec1('ACGT'), vec1('ACTT'), vec1('ACGA')]

In [21]:
# Check pair-wise distances.
for i in range(3):
    for j in range(i + 1, 3):
        print(f'{dist(vecs[i], vecs[j]):.2f}')

2.00
2.00
2.83


In [22]:
# That's good.

## 2. Remember how to use faiss

In this section, I plan to set up and query a vector index.

My plan is to make a list of random vectors, index them, and check that
I can successfully retrieve them even if I make small changes to the
query vector.

In [31]:
# This is the dimension I'll use for the test vectors.
dimn = 10

In [34]:
# Make 1,000 random vectors.
rand_vecs = np.random.rand(1_000, dimn)

In [36]:
# Build the index.
vec_index = faiss.IndexFlatL2(dimn)
vec_index.add(rand_vecs)

In [50]:
# Confirm that exact lookups work.
for _ in range(3):
    idx = random.randint(0, 999)
    distances, indices = vec_index.search(rand_vecs[idx].reshape(1, -1), 5)
    print(idx, indices, distances)

142 [[142 553 512 346 927]] [[0.         0.33740926 0.37946406 0.43927175 0.45978835]]
52 [[ 52 340 118 521 355]] [[0.         0.30508032 0.35074192 0.4079569  0.41953716]]
377 [[377 867 839 638 502]] [[0.         0.39025795 0.51106936 0.5268204  0.54613125]]


In [51]:
# That looks like it's working correctly to me.